In [256]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from numpy.linalg import multi_dot
from scipy.stats import expon


In [257]:
#Reading Regressors.txt
df = pd.read_csv("Regressors.txt", header = None)
df.head

<bound method NDFrame.head of     0         1        2        3         4        5         6         7  \
0   1 -0.637280 -2.53690  1.00570  1.835800  0.74296 -0.900550 -0.385430   
1   1  0.329080 -0.87360  1.51210 -0.723570  0.49154  0.862480 -0.053105   
2   1  0.369910 -1.05260 -0.76142 -0.893250 -0.21182 -0.064187 -0.133520   
3   1 -0.536420  1.20020  0.31715 -1.025000 -0.29137 -1.168800  0.010371   
4   1  0.750650 -0.23990 -0.48102 -0.015687  0.84527 -0.965980 -0.795350   
.. ..       ...      ...      ...       ...      ...       ...       ...   
95  1  0.778860 -0.77748  0.35712 -1.197800 -0.35897 -1.420100  0.853770   
96  1 -0.735240 -1.16190 -1.21380  0.904510  0.24747 -1.406300 -1.143000   
97  1 -0.974290  1.54520 -0.12210  0.786480  2.66050  0.915620 -1.017000   
98  1 -0.018074 -1.40830  0.87636 -0.897840 -0.91976  0.754050  0.013707   
99  1  1.663100 -0.26624  0.07666 -0.966130 -3.03220 -0.145520  0.448210   

          8  
0  -1.01310  
1   1.00120  
2   0.62344  
3

In [258]:
#(I-X(X'X)^-1-X')y
m = np.array(df)
m_trans = m.transpose()
first_mult = multi_dot([m_trans, m])
inverse = np.linalg.inv(first_mult) 
sec_mult = multi_dot([m, inverse, m_trans])
sec_mult.shape
identity_matrix = np.identity(100)
identity_matrix
sub =np.subtract(identity_matrix, sec_mult)

In [259]:
#Reading Observables.txt
df_y = pd.read_csv("Observables.txt", header = None)
df_y.head
y_array = np.array(df_y)
epsilon = multi_dot([sub, y_array])
epsilon[0:5]

array([[-0.19485222],
       [ 1.72042305],
       [ 2.63451544],
       [-0.80304088],
       [-0.29226681]])

In [260]:
# (e(t) - e(t-1))^2
eps_diff = np.diff(epsilon, n=1, axis=0,)
eps_sqr = np.square(eps_diff)
sum1 = np.sum(eps_sqr, axis=0)
sum1


array([192.91561707])

In [261]:
eps_den = np.square(epsilon[:-1])
sum2 = np.sum(eps_den)
sum2

107.72268508774314

In [262]:
test_statistic = list(sum1/sum2)
test_statistic

In [263]:
#Monte Carlo Test
list_tstats = [test_statistic[0]]
for n in range(9999):
  #I made it 100 instead of 1000 because our sample size 100 (100 data points) and the size affects the measurement Durbin-Watson statistics, now results make sense
  ran_exp = np.random.standard_normal(size = 100)
  eps_diff = np.diff(ran_exp, n=1, axis=0,)
  eps_sqr1 = np.square(eps_diff)
  ep_sqr = np.square(ran_exp)
  suma = np.sum(eps_sqr1)
  sumb = np.sum(ep_sqr)
  t_stat = suma/sumb
  list_tstats.append(t_stat)




In [264]:
#rank the simulated data
list_tstats.sort()
list_tstats[0:5]
list_tstats[-5:]

[2.6082145286992557,
 2.609668602127921,
 2.615764475486671,
 2.6520684731700372,
 2.661647853319753]

In [265]:
k1 = int((9999+1)*(0.1/2) + 1)
k2 = int((1-0.1/2)*(9999+1))

In [266]:
rej_c1 = list_tstats[k1-1]

In [267]:
rej_c2 = list_tstats[k2-1]
rej_c2

2.30469562736815

In [268]:
print(rej_c1," ",rej_c2)
print(test_statistic)

1.6558715780809066   2.30469562736815
[1.7908541447664765]


In [269]:
def check(list, val):
  num = 0
  for n in list:
    if n <= val:
      num = num+1
    else:
      num = num
  return num


In [270]:
value = check(list_tstats, test_statistic[0])
value

1663

In [271]:
def check2(list, val):
  num = 0
  for n in list:
    if n >= val:
      num = num+1
    else:
      num = num
  return num

In [272]:
value2 = check2(list_tstats, test_statistic[0])
value2

8338

In [273]:
#Both denominators must be 10000
p_value = 2 *min([value/10000, value2/10000])
p_value

0.3326

## Part C

In [274]:
#Reading True_null.txt
df_tn = pd.read_csv("True_null.txt", header = None)
df_tn.head

<bound method NDFrame.head of         0        1         2         3        4        5        6     \
0  -0.627920 -0.24685  0.257740 -1.660900 -0.47635 -1.11590 -2.08230   
1   1.387200  0.58790 -1.174900  0.136790 -1.84960 -0.59578 -0.26431   
2   2.201100 -0.43099 -0.026149  0.443790  1.19560  0.42235 -0.30351   
3  -1.193200  1.24200  0.258260  1.487800  0.69618 -1.04460 -0.64679   
4  -0.749890  1.76020 -0.442110 -1.684500  0.13206 -1.10050 -2.18150   
..       ...      ...       ...       ...      ...      ...      ...   
95  0.315540  0.67422  0.644480 -1.099900 -0.33078 -0.83251  2.63650   
96 -0.995970  0.23935 -2.097400  0.030075 -0.27806 -0.27771  0.29327   
97 -1.478100 -0.63553 -1.098500 -2.518100 -0.78057 -0.64211 -0.35635   
98  0.541580  1.41560 -0.179180 -1.420500 -0.29322 -1.21950  0.22336   
99  0.060733 -1.90280 -0.464730 -1.552300 -1.33480 -1.31340  0.95880   

        7        8         9     ...      9990     9991     9992      9993  \
0  -1.808000  1.17990 -1.11

In [275]:
def pval(list,val,size):
    return  2*min([check(list,val)/size+1, check2(list,val)/size+1])
    
probs = np.zeros(10000)

#Old Solution

#size = 99
#
#for i in range(10000):   
#    y_array = np.array(df_tn[i])
#    epsilon = multi_dot([sub, y_array])
#    eps_diff = np.diff(epsilon, n=1, axis=0,)
#    eps_sqr = np.square(eps_diff)
#    sum1 = np.sum(eps_sqr, axis=0)
#    eps_den = np.square(epsilon)
#    sum2 = np.sum(eps_den)
#    test_statistic = sum1/sum2
#    list_tstats = [test_statistic]
#    
#    for n in range(size):
#        ran_exp = np.random.standard_normal(size = 100)
#        eps_diff = np.diff(ran_exp, n=1, axis=0,)
#        eps_sqr1 = np.square(eps_diff)
#        ep_sqr = np.square(ran_exp)
#        suma = np.sum(eps_sqr1)
#        sumb = np.sum(ep_sqr)
#        t_stat = suma/sumb
#        list_tstats.append(t_stat)
#        
#    k1 = int((size+1)*(0.1/2) + 1)
#    k2 = int((1-0.1/2)*(size+1))
#    rej_c1 = list_tstats[k1-1]
#    rej_c2 = list_tstats[k2-1]
#    
#    if (test_statistic<rej_c1 or test_statistic>rej_c2):
#        probs[i] = 1
        
        
    



#New Solution

#Rejection check for N columns 
for i in range(10000):
    y_array = np.array(df_tn[i])
    epsilon = multi_dot([sub, y_array])
    
    eps_diff = np.diff(epsilon, n=1, axis=0,)
    eps_sqr = np.square(eps_diff)
    sum1 = np.sum(eps_sqr, axis=0)
    
    eps_den = np.square(epsilon)
    sum2 = np.sum(eps_den)
    
    test_statistic = sum1/sum2
    if(test_statistic<rej_c1 or test_statistic>rej_c2):
        probs[i] = 1
    


#Probability of rejection
alpha = np.mean(probs)
    

In [276]:
print(alpha) 

0.0975


## Part E

In [277]:
df_fn = pd.read_csv("False_null.txt", header = None)
df_fn.head


<bound method NDFrame.head of         0        1        2        3        4        5        6         7     \
0  -0.712300 -0.33122  0.17336 -1.74520 -0.56072 -1.20030 -2.16660 -1.892400   
1   1.244700  0.52166 -1.14020 -0.21226 -1.96170 -0.83584 -0.69764  0.698640   
2   2.450000 -0.32665 -0.25420  0.40134  0.80325  0.25519 -0.44303 -0.992860   
3  -0.703170  1.17660  0.20742  1.56810  0.85683 -0.99354 -0.73540 -1.628000   
4  -0.890530  1.99550 -0.40062 -1.37080  0.30343 -1.29920 -2.32860 -0.979650   
..       ...      ...      ...      ...      ...      ...      ...       ...   
95 -0.062718  0.51397  0.41479 -1.22140 -0.40434 -1.10830  2.56590  1.921100   
96 -1.008500  0.34215 -2.01440 -0.21421 -0.35893 -0.49937  0.80645 -1.292700   
97 -1.679800 -0.56710 -1.50140 -2.56090 -0.85235 -0.74198 -0.19506 -0.143720   
98  0.205610  1.30220 -0.47946 -1.93270 -0.46369 -1.36790  0.18435  0.208850   
99  0.101860 -1.64230 -0.56062 -1.93880 -1.42750 -1.58700  0.99567  0.043056   

        8

In [278]:
probs = np.zeros(10000)
true_coverage = np.zeros(10000)


#Old Solution

#size = 99
#
#for i in range(10000):   
#    y_array = np.array(df_fn[i])
#    epsilon = multi_dot([sub, y_array])
#    eps_diff = np.diff(epsilon, n=1, axis=0,)
#    eps_sqr = np.square(eps_diff)
#    sum1 = np.sum(eps_sqr, axis=0)
#    eps_den = np.square(epsilon)
#    sum2 = np.sum(eps_den)
#    test_statistic = sum1/sum2
#    list_tstats = [test_statistic]
#    
#    for n in range(size):
#        ran_exp = np.random.standard_normal(size = 100)
#        eps_diff = np.diff(ran_exp, n=1, axis=0,)
#        eps_sqr1 = np.square(eps_diff)
#        ep_sqr = np.square(ran_exp)
#        suma = np.sum(eps_sqr1)
#        sumb = np.sum(ep_sqr)
#        t_stat = suma/sumb
#        list_tstats.append(t_stat)
#        
#    k1 = int((size+1)*(0.1/2) + 1)
#    k2 = int((1-0.1/2)*(size+1))
#    rej_c1 = list_tstats[k1-1]
#    rej_c2 = list_tstats[k2-1]
#    
#    if (1-rej_c2/2 <= 0.2 and 0.2 <=1-rej_c1/2):
#        true_coverage[i] = 1
#    if (test_statistic>rej_c1 and test_statistic<rej_c2):
#        probs[i] = 1


#New Solution

#Coverage and Rejection check for N columns
for i in range(10000):
    y_array = np.array(df_fn[i])
    epsilon = multi_dot([sub, y_array])
    
    eps_diff = np.diff(epsilon, n=1, axis=0,)
    eps_sqr = np.square(eps_diff)
    sum1 = np.sum(eps_sqr, axis=0)
    
    eps_den = np.square(epsilon)
    sum2 = np.sum(eps_den)
    
    test_statistic = sum1/sum2
    probs[i] = (test_statistic >= rej_c1 and test_statistic <= rej_c2)
    true_coverage[i] = (1-rej_c2/2 <= 1-test_statistic/2 and 1-test_statistic/2  <=1-rej_c1/2)
    
    

#type-2 error, (not rejecting null while it is false)
beta = np.mean(probs)


In [279]:
# power of the test = (1 - beta)
print(1-beta)
print(np.mean(true_coverage))


0.5217
0.4783
